<a href="https://colab.research.google.com/github/fbeilstein/machine_learning/blob/master/workbook_02_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Run the following block to get helper functions `%%debug_cell_with_pytutor` and `%%embed_into_game`. Explanation follows.

In [0]:
#@title #Black magic (just run it to get few helper functions)

from IPython.core.magic import  Magics, magics_class, cell_magic, line_magic

@magics_class
class Helper(Magics):

  def __init__(self, shell=None,  **kwargs):
    super().__init__(shell=shell, **kwargs)

  @cell_magic
  def debug_cell_with_pytutor(self, line, cell):
    import urllib.parse
    url_src = urllib.parse.quote(cell)
    str_begin = '<iframe width="1000" height="500" frameborder="0" src="https://pythontutor.com/iframe-embed.html#code='
    str_end   = '&cumulative=false&py=3&curInstr=0"></iframe>'
    import IPython
    from google.colab import output
    display(IPython.display.HTML(str_begin+url_src+str_end))

  @cell_magic
  def embed_into_game(self, line, cell):
    games = {'snake': self._snake_game}
    games[line](cell)

  def _snake_game(self, python_code):
    str_begin = '''
<script type="text/javascript" src="https://brython.info/src/brython.js"></script>
<script type="text/javascript" src="https://brython.info/src/brython_stdlib.js"></script>
<script type="text/python3">
    '''
    str_end = '''
from browser import document, svg, timer

class Field:
  def __init__(self, w, h):
    self.rects = [[svg.rect(x=40*i, y=40*j, width=38, height=38, fill='#000000') for i in range(w)] for j in range(h)]
    self.panel = document['panel']
    for row in self.rects:
      for rect in row:
        self.panel <= rect

  def clear(self):
    for row in self.rects:
      for rect in row:
        rect.attrs['fill'] = '#000000'

  def draw_snake(self, snake):
    for i,j in snake:
      (self.rects[j][i]).attrs['fill'] = '#00AA00'

  def draw_apple(self, apple):
    i,j = apple
    (self.rects[j][i]).attrs['fill'] = '#AA0000'

field = Field(15, 15)
snake = Snake(15, 15)
snake.reset()
apple = Apple(15, 15)
apple.reset()

def keyCode(ev):
  ev.stopPropagation()
  snake.set_direction({87:[0,-1], 83:[0,1], 65:[-1,0], 68:[1,0]}[int(ev.keyCode)])

document.onkeydown = keyCode

def evolute():
  snake.move_one_step()
  if snake.head_collision():
    snake.reset()
    apple.reset()
  field.clear()
  field.draw_apple(apple.get_apple())
  field.draw_snake(snake.get_snake())
  head = snake.get_snake()[0]
  a = apple.get_apple()
  if head[0] == a[0] and head[1] == a[1]:
    snake.grow_on_next_step()
    apple.reset()
    
loop = timer.set_interval(evolute, 300)

</script>

<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" 
    width="600" height="600" style="border-style:solid;border-width:1;border-color:#000;">
  <g id="panel">
  </g>
</svg>

<script>
brython(1)
</script>
    '''
    import IPython
    from google.colab import output
    display(IPython.display.HTML(str_begin + python_code + str_end))

## use ipython load_ext mechanism here if distributed
get_ipython().register_magics(Helper)

# Learn to debug

The following code generates list of lists that contain strings of a form 'row,column'.

* **problem 1:** run the script and see its output
* **problem 2:** add at the top of the cell `%%debug_cell_with_pytutor` to run a remote debugger. Use debugger to undestand how the code works.

In [0]:
total_list = []
for i in range(3):
  l = []
  for j in range(3):
    l.append('{},{}'.format(i,j))
  total_list.append(l)
total_list

[['0,0', '0,1', '0,2'], ['1,0', '1,1', '1,2'], ['2,0', '2,1', '2,2']]

#Learn to solve this lecture's problems

first, read through the problem. you will mostly be asked to implement some missing parts of code. Please, do not use `numpy` or any fancy libraries at this stage -- we will need to convert your code to JS later.

In [0]:
%%embed_into_game snake

#Control: letters `w`,`a`,`s`,`d`

import random

class Apple:
  def __init__(self, w, h):
    self.pos = [0, 0]
    self.w = w
    self.h = h
        
  def reset(self):
    self.pos = [random.randint(0, self.w - 1), random.randint(0, self.h - 1)]

  def get_apple(self):
    return self.pos

class Snake:
  def __init__(self, w, h):
    self.pos = []
    self.direction = []
    self.grow = False
    self.w = w
    self.h = h
        
  def move_one_step(self):
    head = [self.pos[0][0] + self.direction[0], self.pos[0][1] + self.direction[1]]
    if not self.grow:
      self.pos = [head] + self.pos[:-1]
    else:
      self.pos = [head] + self.pos
      self.grow = False
    
  def set_direction(self, direction):
    if not self.direction[0] * direction[0] + self.direction[1] * direction[1]:
      self.direction = direction
    
  def get_snake(self):
    return self.pos
    
  def reset(self):
    self.pos = [[10,10], [10,11]]
    self.direction = [0, -1]
    self.grow = False
        
  def grow_on_next_step(self):
    self.grow = True
        
  def head_collision(self):
    head = self.pos[0]
    if head[0] < 0 or head[0] >= self.w:
      return True
    if head[1] < 0 or head[1] >= self.h:
      return True
    if head in self.pos[1:]:
      return True
    return False

snake = Snake(15, 15)
assert isinstance(snake, Snake), "Not instance of snake"
print("ALL TESTS SUCCESSFULLY PASSED")